1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [76]:
import pandas as pd
import random as rnd

file_name = "./data/data_000637.txt"

#creo un numero random tra 10000 e il numero massimo di righe
tmp = pd.read_csv(file_name)
N=rnd.randint(10000,len(tmp))
print(f"Total number of rows: {len(tmp)}")
print(f"N: {N}")

data = pd.read_csv(file_name, nrows=N) #leggo N righe del file
data


Total number of rows: 1310720
N: 1098560
Total numbers of rows: 1098560


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
1098555,1,1,1,3869209698,2983,25
1098556,1,1,2,3869209698,2986,17
1098557,1,0,44,3869209698,3110,10
1098558,1,0,64,3869209698,3115,23


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [77]:
#x= massimo della colonna BX_COUNTER
x=data["BX_COUNTER"].max()

print("The max of BX_COUNTER is:", x)

The max of BX_COUNTER is: 3563


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [78]:
import numpy as np
import datetime

#calcolo i tempi per ogni riga
abs_time=[]
print("Wait...")
for i,row in data.iterrows():
    tmp_time= (row["TDC_MEAS"]*25/30+ row["BX_COUNTER"]*25+ row["ORBIT_CNT"]*x*25)
    abs_time.append(tmp_time)

#aggiungo una nuova colonna con i tempi per ogni riga
data['ABS_TIME'] = abs_time

#stampo le prime venti righe per controllare
m = data.head(10)
print(f"\nFirst 10 rows:\n{m}")

#converto la nuova colonna in time-series:

time_series=pd.to_datetime(data["ABS_TIME"], unit="ns")
print(f"\nFirst 10 rows of Time-Series:\n{time_series.head(10)}")

Wait...

First 10 rows:
   HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS      ABS_TIME
0     1     0          123  3869200167        2374        26  3.446490e+14
1     1     0          124  3869200167        2374        27  3.446490e+14
2     1     0           63  3869200167        2553        28  3.446490e+14
3     1     0           64  3869200167        2558        19  3.446490e+14
4     1     0           64  3869200167        2760        25  3.446490e+14
5     1     0           63  3869200167        2762         4  3.446490e+14
6     1     0           61  3869200167        2772        14  3.446490e+14
7     1     0          139  3869200167        2776         0  3.446490e+14
8     1     0           62  3869200167        2774        21  3.446490e+14
9     1     0           60  3869200167        2788         7  3.446490e+14

First 10 rows of Time-Series:
0   1970-01-04 23:44:09.004934896
1   1970-01-04 23:44:09.004934897
2   1970-01-04 23:44:09.004939373
3   1970-01-04 23:

4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [113]:
#leggo l'intero file
data = pd.read_csv(file_name)

#calcolo i tempi per ogni riga
abs_time=[]
print("Wait...")
for i,row in data.iterrows():
    tmp_time= (row["TDC_MEAS"]*25/30+ row["BX_COUNTER"]*25+ row["ORBIT_CNT"]*x*25)
    abs_time.append(tmp_time)

#aggiungo una nuova colonna con i tempi per ogni riga
data['ABS_TIME'] = pd.to_timedelta(abs_time)

print(data.head(10))

Wait...
   HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS  \
0     1     0          123  3869200167        2374        26   
1     1     0          124  3869200167        2374        27   
2     1     0           63  3869200167        2553        28   
3     1     0           64  3869200167        2558        19   
4     1     0           64  3869200167        2760        25   
5     1     0           63  3869200167        2762         4   
6     1     0           61  3869200167        2772        14   
7     1     0          139  3869200167        2776         0   
8     1     0           62  3869200167        2774        21   
9     1     0           60  3869200167        2788         7   

                   ABS_TIME  
0 3 days 23:44:09.004934896  
1 3 days 23:44:09.004934897  
2 3 days 23:44:09.004939373  
3 3 days 23:44:09.004939490  
4 3 days 23:44:09.004944545  
5 3 days 23:44:09.004944578  
6 3 days 23:44:09.004944836  
7 3 days 23:44:09.004944925  
8 3 days 23:44:09

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [114]:
count = data.groupby(['TDC_CHANNEL']).count()
n=3  
noisy_ch=count.sort_values(by="HEAD",ascending=True).iloc[0:n] ##selecting how many channels to show
print(noisy_ch)

             HEAD  FPGA  ORBIT_CNT  BX_COUNTER  TDC_MEAS  ABS_TIME
TDC_CHANNEL                                                       
129            37    37         37          37        37        37
137            68    68         68          68        68        68
138            70    70         70          70        70        70


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.